In [1]:
from fastai.vision.all import *
import torchaudio
import librosa
from nnAudio.features.cqt import CQT1992v2
from scipy.signal import tukey


C:\Users\anaconda\envs\Pytorch_19\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class CFG:
    apex=False
    debug=False
    print_freq=100
    num_workers=4
    model_name='tf_efficientnet_b7_ns'
    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']
    epochs=3
    #factor=0.2 # ReduceLROnPlateau
    #patience=4 # ReduceLROnPlateau
    #eps=1e-6 # ReduceLROnPlateau
    T_max=3 # CosineAnnealingLR
    #T_0=3 # CosineAnnealingWarmRestarts
    lr=1e-4
    min_lr=1e-6
    batch_size=48
    weight_decay=1e-6
    gradient_accumulation_steps=1
    max_grad_norm=1000
    qtransform_params={"hop_length": 1024, "n_bins": 64,'fmax':None}    
    seed=42
    target_size=1
    target_col='target'
    n_fold=5
    trn_fold=[0] # [0, 1, 2, 3, 4]
    train=True
    grad_cam=False


In [3]:
df_train = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/train.csv')
df_test = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/test.csv')
submission = pd.read_csv(r'C:\Users\Kaggle\Pog_Music_Classification/sample_submission.csv')

In [4]:
train_path = Path('C:/Users/Kaggle/Pog_Music_Classification/train/')
test_path = Path('C:/Users/Kaggle/Pog_Music_Classification/test/')

In [5]:
print(df_train.shape,df_test.shape)

(19922, 5) (5078, 3)


In [6]:
df_train.head()

,song_id,filename,filepath,genre_id,genre
0,10150,010150.ogg,train/010150.ogg,7,Instrumental
1,7358,007358.ogg,train/007358.ogg,2,Punk
2,20573,020573.ogg,train/020573.ogg,5,Folk
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic
4,16662,016662.ogg,train/016662.ogg,1,Rock


In [7]:
import glob
valid_filenames = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/train/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames.append(fnames.split('.')[0]+'.ogg')

In [8]:
import glob
valid_filenames_test = []
for i in glob.glob(r'C:/Users/Kaggle/Pog_Music_Classification/resampled_16k/test/*.npy'):
    fnames = (i.split('\\')[1])
    valid_filenames_test.append(fnames.split('.')[0]+'.ogg')

In [9]:
df_train = df_train[df_train['filename'].isin(valid_filenames)].reset_index(drop=True)
df_test_valid = df_test[df_test['filename'].isin(valid_filenames_test)].reset_index(drop=True)

In [10]:
### Some files giving exception in cqt
df_train = df_train[df_train['song_id']!=17400].reset_index(drop=True)

In [11]:
print(df_train.shape,df_test.shape)

(19908, 5) (5078, 3)


### Upsample minority classes

In [12]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [13]:
ExtremeMinority = [18]
SomeMinority = [17]
LowMinority = [14,15,16]

# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(ExtremeMinority)].sample(n=37,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(SomeMinority)].sample(n=42,replace=True,random_state=42)],0).reset_index(drop=True)
# df_train = pd.concat([df_train,df_train[df_train['genre_id'].isin(LowMinority)].sample(n=50,replace=True,random_state=42)],0).reset_index(drop=True)

In [14]:
df_train.tail()#,df_train.shape

,song_id,filename,filepath,genre_id,genre
19903,9337,009337.ogg,train/009337.ogg,0,Electronic
19904,8340,008340.ogg,train/008340.ogg,4,Hip-Hop
19905,16248,016248.ogg,train/016248.ogg,4,Hip-Hop
19906,11875,011875.ogg,train/011875.ogg,2,Punk
19907,6579,006579.ogg,train/006579.ogg,1,Rock


In [15]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

#### condense bottom 8 classes into single class

In [16]:
# df_train['genre_id'] = df_train['genre_id'].apply(lambda x: 5 if x>=5 else x).values

In [17]:
# df_train['genre_id'].value_counts()

### Create Folds

In [18]:
df_train['fold'] = 0

In [19]:
from sklearn.model_selection import StratifiedKFold
N_folds = 5
kf = StratifiedKFold(5,random_state=42,shuffle=True)
fld = 0
for i,(_,val_id) in enumerate(kf.split(df_train['filename'],df_train['genre_id'])):
    df_train.loc[val_id,'fold'] = int(fld)
    fld += 1

In [20]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [21]:
df_test.head()

,song_id,filename,filepath
0,7072,007072.ogg,test/007072.ogg
1,10207,010207.ogg,test/010207.ogg
2,20008,020008.ogg,test/020008.ogg
3,10924,010924.ogg,test/010924.ogg
4,21896,021896.ogg,test/021896.ogg


#### Channel wise max: 2.7517, 2.7950

#### Dataset

In [22]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision.transforms import Compose
import torchvision

tfms = Compose(
#     [torchvision.transforms.Resize((128,128))])
    [torchvision.transforms.CenterCrop((160,1072))])


In [23]:
import torch_audiomentations as tA

def get_transforms(*, data):
    
    if data == 'train':
        return tA.Compose(
                transforms=[
#                      tA.ShuffleChannels(p=0.1,mode="per_example",p_mode="per_example", sample_rate= 16000),
                    tA.PeakNormalization(p=0.5,apply_to='only_too_loud_sounds',mode="per_example",p_mode="per_example",sample_rate=16000),
#                     tA.BandPassFilter(),
                    tA.TimeInversion(p=0.5,p_mode="per_example",sample_rate=16000),
#                     tA.PitchShift()
#                      tA.AddColoredNoise(p=0.1,mode="per_channel",p_mode="per_channel", sample_rate=16000,max_snr_in_db = 15),
                     tA.Shift(p=0.5,mode="per_example",p_mode="per_example", sample_rate=16000,
                              max_shift=0.025, min_shift=-0.025),
                ])

    elif data == 'valid':
        return tA.Compose([
        ])

In [24]:
class POGFiles(torch.utils.data.Dataset):
    
    def __init__(self, df, 
                 train = True,
                 augmentations = None,
                 normalize=False,
                 ch0Max=2.7517,
                 ch1Max=2.7950,
                 HOP_LEN = 1024,
                 N_FFT=1024,
                 n_mels=224,
                 transform=tfms,
                 audiotfms=None,
                 applyTukey=False,
                 quantizeData=False,
                 MakeThreeChannel= False,
                 bins_per_octave = 24,
                 n_bins = 64
                ):
        
        
        self.df = df
        self.train = train
        self.ch0Max = ch0Max
        self.ch1Max = ch1Max        
        self.N_FFT = N_FFT
        self.n_bins = n_bins
        self.bins_per_octave = bins_per_octave
        self.HOP_LEN = HOP_LEN
        self.transform = transform
        self.n_mels= n_mels
        self.audiotfms = audiotfms
        self.normalize = normalize
        self.MakeThreeChannel = MakeThreeChannel
        
    def __len__(self):
        return len(self.df)
    
    def load_file(self, filename):
        if self.train:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/448_160/train/{filename}')
        else:
            spec = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/spec/448_160/test/{filename}')
            
        return spec


    def __getitem__(self, index):
        row  = self.df.iloc[index]
        
        samples = self.load_file(row.filename.split('.')[0]+'.npy')
        samples = torch.from_numpy(samples).float()
        
        if self.transform is not None:
            samples = self.transform(samples)
            
        if self.train:
            label = torch.tensor(row.genre_id,dtype=torch.long)
            return samples, label
        else:

            return samples

In [25]:
# a,b = next(iter(POGFiles(df_train)))
# a.shape

In [26]:
a,b = next(iter(POGFiles(df_train,transform=None)))
#` a,b = next(iter(POGFiles(df_train,transform=None,n_mels=128,HOP_LEN=576,N_FFT=512)))
a.shape

torch.Size([1, 160, 1072])

#### FastAI model

In [27]:
df_train['genre_id'].nunique()

19

In [28]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [29]:
import timm
timm.list_models()

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'botnet50ts_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_tiny_hnf',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_x

In [30]:
# timm.create_model('resnest50d',in_chans=1)

In [31]:
# timm.create_model('resnext50_32x4d',in_chans=1).conv1.stride

In [32]:
df_train['genre_id'].value_counts()

1     3095
0     3071
2     2582
3     1800
4     1756
5     1214
6     1181
7     1044
8      945
9      814
10     796
11     495
12     408
13     306
14     142
15      94
16      94
17      58
18      13
Name: genre_id, dtype: int64

In [33]:
df_train.head()

,song_id,filename,filepath,genre_id,genre,fold
0,10150,010150.ogg,train/010150.ogg,7,Instrumental,4
1,7358,007358.ogg,train/007358.ogg,2,Punk,3
2,20573,020573.ogg,train/020573.ogg,5,Folk,2
3,11170,011170.ogg,train/011170.ogg,12,Old-Time / Historic,3
4,16662,016662.ogg,train/016662.ogg,1,Rock,3


In [34]:
NCLASS = df_train['genre_id'].nunique()

In [35]:
NCLASS

19

In [36]:
def run():
    oof = df_train.copy()
    oversampleTrain = False
    
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'resnest50d_MonoSpec_Hop448_Mels160'
        kernel_type_prev = 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit = CrossEntropyLossFlat() #LabelSmoothingCrossEntropy(0.025) #CrossEntropyLossFlat() #FocalLoss()
        batch_size = 22
        n_epochs = 7

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            transform=tfms
                           )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            transform=tfms)

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        
        
        model = timm.create_model('resnest50d', pretrained=False, in_chans=1)#get_model(resnet34)
        model.conv1[0].stride = (1,1)
        model.fc = nn.Linear(2048,NCLASS)

    
        f1_score = F1Score(average="micro")
        
        
        
        ### MixUp() in callbacks
#        ReduceLROnPlateau(monitor='valid_loss', min_delta=0.1,patience=2,min_lr=1e-7)
# GradientAccumulation (n_acc = 2)
        
        learn = Learner(dls, model,opt_func=ranger,loss_func=crit,metrics=[f1_score],cbs=[SaveModelCallback('f1_score', every_epoch=True),
                                                                                          CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')]).to_fp16()
        
#         learn.load()
        weights = torch.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_prev}/fold_{fold_num}.pth')['model']
        learn.model.load_state_dict(weights,strict=False)

#         learn.fit_one_cycle(n_epochs, 1e-4, wd=1e-03)
        learn.fit_flat_cos(n_epochs, 1e-2, wd=1e-03)
#         learn = learn.to_fp16()
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                           transform=tfms,
                           train=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [37]:
if __name__=='__main__':
    run()

*****************************************
Training Fold 0
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.407457,1.605660,0.480161,11:34
1,1.348731,1.788739,0.415118,11:07
2,1.303841,1.527115,0.504018,11:07
3,1.254887,1.814996,0.455048,11:11
4,1.207071,1.518897,0.506781,11:06
5,1.137987,1.607901,0.488950,11:53
6,1.021351,1.324396,0.570819,12:17


*****************************************
Training Fold 1
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.392302,1.507782,0.504771,11:23
1,1.344708,1.515236,0.502260,11:17
2,1.307117,1.447003,0.521346,11:15
3,1.275418,2.010076,0.379458,11:17
4,1.250168,2.096886,0.363887,11:23
5,1.164981,1.449420,0.535660,11:23
6,1.022967,1.312910,0.572577,27:20


*****************************************
Training Fold 2
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.432918,1.526018,0.491713,13:28
1,1.367529,1.544734,0.483928,11:14
2,1.330301,2.214098,0.314415,11:14
3,1.279581,2.730974,0.262431,11:15
4,1.250956,1.777519,0.424661,11:21
5,1.163013,3.106297,0.250377,11:16
6,1.032780,1.341175,0.571572,11:14


*****************************************
Training Fold 3
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.410006,1.732262,0.464456,11:23
1,1.352090,1.567985,0.484803,11:14
2,1.311796,1.412133,0.520723,11:18
3,1.271240,1.558853,0.492841,11:15
4,1.234355,1.473826,0.525245,11:13
5,1.172471,1.674614,0.437076,11:13
6,1.043755,1.346699,0.564933,11:15


*****************************************
Training Fold 4
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.385957,2.035295,0.354182,11:14
1,1.343646,1.600174,0.490580,11:14
2,1.290671,1.857674,0.431801,11:13
3,1.255904,1.581935,0.504647,11:13
4,1.219337,1.691043,0.468224,11:13
5,1.142838,1.767836,0.432555,11:15
6,1.027535,1.338554,0.564682,11:14


In [38]:
submission

,song_id,genre_id
0,7072,0
1,10207,0
2,20008,0
3,10924,0
4,21896,0
...,...,...
5073,6427,0
5074,16903,0
5075,1731,0
5076,12871,0


In [39]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [40]:
import time

#### Fintune even more

In [41]:
def run():
    oof = df_train.copy()
    oversampleTrain = False
    
    for fold_num in [0,1,2,3,4]:
        print('*****************************************')
        print(f'Training Fold {fold_num}')
        print('*****************************************')

    #with IPyExperimentsPytorch() as exp:
        kernel_type = 'resnest50d_MonoSpec_Hop448_Mels160_FTune'
        kernel_type_prev = 'resnest50d_MonoSpec_Hop448_Mels160'
        OUTPUT_DIR = f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/'
        if not os.path.exists(OUTPUT_DIR):
            os.makedirs(OUTPUT_DIR)

            
        crit = CrossEntropyLossFlat() #LabelSmoothingCrossEntropy(0.025) #CrossEntropyLossFlat() #FocalLoss()
        batch_size = 22
        n_epochs = 7

        training_fold = df_train.query(f'fold!={fold_num}').reset_index(drop=True, inplace=False)
        
        if oversampleTrain:
            print('--------- Oversampling training dataset -----------')
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(ExtremeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(SomeMinority)].sample(n=150,replace=True,random_state=42)],0).reset_index(drop=True)
            training_fold = pd.concat([training_fold,training_fold[training_fold['genre_id'].isin(LowMinority)].sample(n=100,replace=True,random_state=42)],0).reset_index(drop=True)
                
        train_ds = POGFiles(training_fold,
                            transform=tfms
                           )

        validation_fold = df_train.query(f'fold=={fold_num}').reset_index(drop=True, inplace=False)
        valid_ds = POGFiles(validation_fold,
                            transform=tfms)

        print(f'- Training samples: {len(train_ds)}\n- Validation Samples : {len(valid_ds)}')

        bs = batch_size
        train_dl = torch.utils.data.DataLoader(train_ds, batch_size=bs, num_workers=0,pin_memory=False)
        valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=bs*2, num_workers=0,shuffle=False,pin_memory=False)

        dls = DataLoaders(train_dl, valid_dl)
        
        
        model = timm.create_model('resnest50d', pretrained=False, in_chans=1)#get_model(resnet34)
        model.conv1[0].stride = (1,1)
        model.fc = nn.Linear(2048,NCLASS)

    
        f1_score = F1Score(average="micro")
        
        
        
        ### MixUp() in callbacks
#        ReduceLROnPlateau(monitor='valid_loss', min_delta=0.1,patience=2,min_lr=1e-7)
# GradientAccumulation (n_acc = 2)
        
        learn = Learner(dls, model,loss_func=crit,metrics=[f1_score],cbs=[SaveModelCallback('f1_score', every_epoch=True),
                                                                                          CSVLogger(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/{fold_num}logs.csv')]).to_fp16()
        
#         learn.load()
        weights = torch.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_prev}/fold_{fold_num}.pth')['model']
        learn.model.load_state_dict(weights,strict=False)

        learn.fit_one_cycle(n_epochs, 1e-4, wd=1e-03)
#         learn.fit_flat_cos(n_epochs, 1e-4, wd=1e-03)
#         learn = learn.to_fp16()
        learn.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        learn = learn.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}')
        
        learn.model.eval()
        test_df = df_test
        test_ds = POGFiles(df_test_valid,
                           transform=tfms,
                           train=False
                           )

        test_ds.input_path = Path(test_path)

        bs = batch_size
        test_dl = torch.utils.data.DataLoader(test_ds, batch_size=bs*2, num_workers=0, shuffle=False)
        
        preds = []
        probs = []
        for xb  in progress_bar(test_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            probs.append(torch.softmax(output.float(),1).squeeze().cpu())
            preds.append(torch.argmax(output.float(),1).squeeze().cpu())
        
        oof_ = []
        for xb,_  in progress_bar(valid_dl):
            with torch.no_grad():output = learn.model(xb.cuda())
            oof_.append(torch.softmax(output.float(),1).squeeze().cpu())
        
        oof_ = [t.detach().numpy() for t in oof_]
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/test_fold_{fold_num}_probs.npy',np.array(probs))
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',oof_)
        np.save(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',np.array(validation_fold.song_id))        

        preds = torch.cat(preds)
        sample_df = df_test_valid.copy()
        sample_df['target'] = preds
        sample_df.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fold_num}.csv', index=False)

In [42]:
if __name__=='__main__':
    run()

*****************************************
Training Fold 0
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.006392,1.322383,0.573079,11:14
1,0.985888,1.329251,0.571321,11:12
2,0.965288,1.335465,0.573079,11:12
3,0.949938,1.341175,0.574083,11:13
4,0.938507,1.345523,0.575339,11:13
5,0.931381,1.348506,0.573581,11:13
6,0.928729,1.349184,0.573832,11:16


*****************************************
Training Fold 1
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.007252,1.312649,0.573581,11:35
1,0.986189,1.318517,0.577097,11:32
2,0.966476,1.324990,0.576846,11:32
3,0.951978,1.331213,0.575088,11:33
4,0.941840,1.334895,0.573832,11:32
5,0.935744,1.336731,0.573832,12:58
6,0.933272,1.337146,0.573330,12:06


*****************************************
Training Fold 2
*****************************************
- Training samples: 15926
- Validation Samples : 3982


epoch,train_loss,valid_loss,f1_score,time
0,1.015442,1.336428,0.571823,11:32
1,0.993451,1.341583,0.570819,11:32
2,0.969682,1.348588,0.572325,11:32
3,0.952428,1.353650,0.572325,11:32
4,0.940318,1.356949,0.570065,11:32
5,0.932964,1.358564,0.571572,11:32
6,0.930082,1.358863,0.572325,11:32


*****************************************
Training Fold 3
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.029715,1.343011,0.567194,11:33
1,1.008305,1.344524,0.567697,11:32
2,0.984468,1.349844,0.568953,11:32
3,0.967594,1.355226,0.569957,11:32
4,0.956583,1.358944,0.570460,11:33
5,0.949933,1.360276,0.570460,11:32
6,0.947121,1.360566,0.570460,11:32


*****************************************
Training Fold 4
*****************************************
- Training samples: 15927
- Validation Samples : 3981


epoch,train_loss,valid_loss,f1_score,time
0,1.014344,1.337584,0.571213,11:32
1,0.997532,1.344472,0.573223,11:32
2,0.978794,1.350116,0.570962,11:32
3,0.964567,1.354523,0.570460,11:32
4,0.954132,1.357331,0.570460,11:33
5,0.947320,1.358697,0.569204,11:33
6,0.944215,1.359080,0.568953,11:33


In [ ]:
#### Finetune even more

### OOF

In [52]:
OOFOut = pd.DataFrame()
for fld in range(5):
    fold_num = fld
    kernel_type= 'resnest50d_MonoSpec_Hop448_Mels160_FTune'
    OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
    ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
    OOF = ([item for sublist in OOF for item in sublist])
    OOF = pd.DataFrame(OOF)
    OOF['ID'] = ID
    OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)


In [53]:
OOFOut['ID']

0        10150
1        11198
2        13166
3        12275
4        24102
         ...  
19903     2609
19904    13023
19905     2022
19906     9337
19907     6579
Name: ID, Length: 19908, dtype: int64

In [54]:
OOFOut.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,ID
19903,0.134812,0.027021,0.001354,0.064673,0.080743,0.001413,0.000116,0.044586,0.023289,0.575484,0.002587,0.000030,3.612936e-08,0.010149,1.020929e-05,2.929641e-02,6.865471e-08,0.001729,2.708104e-03,2609
19904,0.013989,0.022497,0.003951,0.226010,0.001761,0.242410,0.000005,0.076850,0.039501,0.299509,0.000673,0.000310,2.127126e-07,0.045991,8.591344e-03,1.195261e-04,1.323815e-04,0.017668,3.013018e-05,13023
19905,0.001115,0.685158,0.297828,0.000195,0.000007,0.005261,0.000001,0.002507,0.007613,0.000134,0.000008,0.000001,1.606740e-10,0.000016,1.139533e-04,9.147151e-08,1.301795e-08,0.000040,5.136964e-13,2022
19906,0.539279,0.001800,0.000308,0.014296,0.187414,0.000150,0.199204,0.003199,0.009487,0.000134,0.044584,0.000002,3.979170e-10,0.000017,7.313076e-07,2.619502e-05,1.137097e-07,0.000090,9.346804e-06,9337
19907,0.580139,0.007325,0.000882,0.076510,0.039454,0.002582,0.002398,0.101512,0.005291,0.001010,0.182231,0.000091,1.387177e-08,0.000499,4.961441e-07,2.407738e-06,3.609706e-06,0.000068,1.894773e-06,6579


In [55]:
OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')

In [56]:
OOFOut.head()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,ID,song_id,genre_id
0,0.087290,0.055174,0.012636,0.547353,3.257450e-02,0.052874,4.464758e-04,1.187350e-01,0.057253,0.006194,...,4.371709e-08,1.686004e-03,7.552167e-06,9.941575e-06,5.433745e-03,7.112910e-04,1.794412e-05,10150,10150,7
1,0.001830,0.000431,0.000052,0.000069,1.309610e-07,0.000082,9.497234e-08,5.832285e-06,0.000187,0.995651,...,3.228505e-08,1.595373e-03,3.448425e-05,7.166831e-06,1.658219e-08,4.989763e-05,1.625935e-06,11198,11198,9
2,0.881258,0.040783,0.001718,0.004240,1.138753e-03,0.000090,7.008937e-03,1.131815e-02,0.012124,0.029696,...,2.537065e-09,3.917913e-03,8.159511e-06,9.519067e-06,2.912638e-10,1.202548e-07,3.251076e-09,13166,13166,0
3,0.007851,0.000044,0.000303,0.000120,9.896476e-01,0.000008,3.138293e-05,3.969411e-07,0.001043,0.000460,...,7.555823e-14,3.678345e-09,6.407751e-10,4.801646e-04,3.388639e-08,2.862558e-08,2.063116e-06,12275,12275,4
4,0.026550,0.070271,0.003049,0.686081,4.763220e-04,0.058910,1.801207e-03,8.459811e-02,0.008337,0.000137,...,6.382243e-06,1.406968e-03,1.112011e-05,1.005602e-08,4.131449e-02,7.870882e-05,4.809337e-07,24102,24102,3


In [57]:
OOFOut['genre_id'].max()

18

In [58]:
OOF_preds=OOFOut.iloc[:,:19].idxmax(1)


In [59]:
OOF_preds

0        3
1        9
2        0
3        4
4        3
        ..
19903    9
19904    9
19905    1
19906    0
19907    0
Length: 19908, dtype: int64

In [60]:
from sklearn.metrics import f1_score

f1_score(OOFOut['genre_id'],OOF_preds,average="micro")

0.5719308820574643

In [61]:
pd.crosstab(OOF_preds,OOFOut['genre_id'])

genre_id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
row_0,,,,,,,,,,,,,,,,,,,
0,1962,140,42,154,255,18,349,112,139,85,383,1,0,22,1,30,0,3,5
1,113,1583,762,161,24,124,15,113,221,38,15,4,0,31,30,10,2,10,0
2,31,725,1572,25,3,18,1,16,60,10,2,0,0,2,8,3,2,0,0
3,169,173,67,997,22,62,39,158,93,22,93,20,4,41,1,1,26,4,3
4,198,20,14,32,1361,6,20,6,34,21,9,0,0,0,0,9,2,6,1
5,19,136,18,61,4,817,2,35,94,37,4,8,2,10,24,2,6,11,0
6,202,10,5,35,19,1,715,8,9,4,29,1,0,3,0,1,0,0,0
7,96,80,7,104,9,46,9,473,26,5,73,51,0,26,1,4,1,1,1
8,69,106,41,36,20,46,3,13,189,22,7,3,1,6,13,3,2,4,2


### Mode based sub

In [62]:
kernel_type

'resnest50d_MonoSpec_Hop448_Mels160_FTune'

In [63]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/fold_{fld}.csv')
    out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [64]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode.csv',index=False)

### Mode based sub

In [65]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['resnest50d_MonoSpec_Hop448_Mels160_FTune',
                    'resnest50d_MonoSpec_Hop448_Mels160',
                    ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [66]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_2bestmodels.csv',index=False)

### Mode based sub: check OOF Score

In [76]:
out = pd.DataFrame()
for kernels in ['resnest50d_MonoSpec_Hop448_Mels160_FTune','resnest50d_MonoSpec_Hop448_Mels160',
                'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128','tf_efficientnet_b1_MonoSpec_Hop512_Mels128',
               ]:
    OOFOut = pd.DataFrame()
    for fld in range(5):
        fold_num = fld
        kernel_type= kernels
        OOF = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_probs.npy',allow_pickle=True)
        ID = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/oof_fold_{fold_num}_fnames.npy',allow_pickle=True)
        OOF = ([item for sublist in OOF for item in sublist])
        OOF = pd.DataFrame(OOF)
        OOF['ID'] = ID
        OOFOut = pd.concat([OOF,OOFOut],0).reset_index(drop=True)

    OOFOut = pd.merge(OOFOut,df_train[['song_id','genre_id']],left_on='ID',right_on='song_id',how='left')
    OOF_preds=OOFOut.iloc[:,:19].idxmax(1)
    out = pd.concat([out,OOF_preds],1)
    out['genre_id'] = OOFOut['genre_id']
    
out['preds'] = out.drop('genre_id',1).mode(1)[0].astype(int)
f1_score(out['genre_id'],out['preds'],average="micro")

0.5762005224030541

### Mode based sub

In [80]:
kernel_type ='resnest50d_MonoSpec_Hop448_Mels160_FTune'

In [81]:
out = submission.copy().drop('genre_id',1)
for fld in range(5):
    for kernels in ['resnest50d_MonoSpec_Hop448_Mels160_FTune','resnest50d_MonoSpec_Hop448_Mels160',
                'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128','tf_efficientnet_b1_MonoSpec_Hop512_Mels128'
                    ]:
        fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernels}/fold_{fld}.csv')
        out = pd.merge(out,fname[['song_id','target']],on='song_id',how='left')
    
submission['genre_id']=out.drop('song_id',1).mode(1)[0]

In [82]:
submission.loc[submission['song_id']==22612,'genre_id'] = 1
submission.loc[submission['song_id']==24013,'genre_id'] = 0
submission['genre_id'] = submission['genre_id'].astype(int)
submission.to_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type}/5Fold_mode_ens_4bestmodels.csv',index=False)

In [83]:
kernel_type

'resnest50d_MonoSpec_Hop448_Mels160_FTune'

### PL

In [ ]:
kernel_type_upd = 'resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128' 

In [ ]:
fname = pd.read_csv(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_upd}/fold_{fld}.csv')
fname

In [ ]:
preds_out = pd.DataFrame(np.zeros((5076,19)))
for fld in range(5):
    probs = np.load(f'C:/Users/Kaggle/Pog_Music_Classification/outputs/{kernel_type_upd}/test_fold_{fld}_probs.npy',allow_pickle=True)
    p = pd.DataFrame([item.numpy().ravel() for sublist in probs for item in sublist])
    preds_out += p.values/5

In [ ]:
preds_out.shape

In [ ]:
preds_out['song_id'] = fname['song_id']

In [ ]:
preds_out.head()

In [ ]:
preds_out[(preds_out.iloc[:,:19].values > 0.85).sum(1)>0]

In [ ]:
PL = preds_out[(preds_out.iloc[:,:19].values > 0.85).sum(1)>0].reset_index(drop=True)

In [ ]:
PL.to_csv(r'C:\Users\Kaggle\Pog_Music_Classification\outputs\resnet50d_MIXUP_finetune_MonoSpec_Hop512_Mels128\PL.csv')

In [ ]:
PL.head()

In [ ]:
np.argmax(PL.iloc[:,:-1].values,1)

### End ###